In [ ]:
#| default_exp netcdf_to_csv

# Open Refine CSV (WIP)

***

## Packages import

In [ ]:
#| export
from pathlib import Path # This module offers classes representing filesystem paths
import xarray as xr
from netCDF4 import Dataset
import pandas as pd
import xarray as xr
import numpy as np
from marisco.callbacks import (Callback, Transformer,
                               EncodeTimeCB, SanitizeLonLatCB)
import fastcore.all as fc # package that brings fastcore functionality, see https://fastcore.fast.ai/.
from cftime import num2pydate 
from marisco.configs import cfg, cdl_cfg, nuc_lut_path, unit_lut_path, detection_limit_lut_path, species_lut_path, bodyparts_lut_path, sediments_lut_path
from marisco.serializers import OpenRefineCsvEncoder
from functools import reduce,partial

Get the current working directory (cwd). . 

In [ ]:
#|eval: false
Path.cwd()

Path('/home/marisco/downloads/marisco/nbs/handlers')

## Transform data

### Reshape: wide to long

In [ ]:
#| export
class ReshapeWideToLong(Callback):
    "Convert data from wide to long with renamed columns."
    def __init__(self, columns='nuclide', values=['value']):
        fc.store_attr()
        # Retrieve all possible suffixes vars (e.g '_unc', '_dl', ...) from configs
        suff_cfg = [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]
        # Retrieve all possible nuclides
        nucs_cfg = pd.read_excel(nuc_lut_path())['nc_name'].to_list()
        nucs_cfg = [x for x in nucs_cfg if str(x) != 'nan'] # remove 'nan' from nuclide list
        # Retrieve all possible vars thats are not in vars suffixes
        self.vars_cfg=[x['name'] for var_key in cdl_cfg()['vars'].keys() for x in cdl_cfg()['vars'][var_key].values() if var_key != 'suffixes']
        # combine all possible nuclides with its suffixes.    
        value_name='value'
        derived_nucs_cols={value_name:nucs_cfg}     
        for suf in suff_cfg:
            derived_nucs_cols[suf]= [str(nuc)+str(suf) for nuc in nucs_cfg]
        self.derived_nucs_cols=derived_nucs_cols
           
    def melt(self, df):
        # Among all possible 'self.derived_nuc_cols' include the ones present in df.
        derived_nucs_cols={}
        for key,derived_nuc_cols in self.derived_nucs_cols.items():
            derived_nuc_cols = [col for col in derived_nuc_cols if col in df.columns]
            if derived_nuc_cols:
                derived_nucs_cols[key] = derived_nuc_cols
        
        # Among all possible 'self.vars_cfg' include the ones present in df.
        vars_cfg = [var for var in self.vars_cfg if var in df.columns]
        
        # Melt cols included in self.derived_nucs_cols        
        df=df.reset_index()  # Reset the index so 'sample' can be used with id_vars
        nuc_dfs={}

        for key,val in derived_nucs_cols.items():
            # Transpose nuclide_cols
            df_t=pd.melt(frame=df, id_vars=vars_cfg+['sample'], value_vars=val, var_name='nuclide', value_name=key)
            # Remove the key from thr nuclide columns (e.g '_unit', '_dl', etc.).
            df_t['nuclide']=df_t['nuclide'].str.replace(key, '')
            # Keep rows where 'key' value is not nan
            df_t=df_t[df_t[key].notna()]
            nuc_dfs[key]=df_t    
            
        # Merge dfs created from melt. 
        combine_on= vars_cfg + ['sample'] + ['nuclide']
        merged_df=reduce(lambda df1, df2: pd.merge(df1, df2,  how='left', left_on= combine_on, right_on = combine_on), nuc_dfs.values())
        
        # Keep rows where either value_name (i.e.Activity or MDA ) is not 'nan'.
        merged_df = merged_df[merged_df[['value']].notna().any(axis=1)]
                
        return (merged_df)
    
    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            tfm.dfs[grp] = self.melt(tfm.dfs[grp])


In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong()])
tfm()

{'seawater':              lon        lat  smp_depth  tot_depth        time  sample  \
 0      14.199800  54.006001        0.0       11.0  1497744000      78   
 1      14.202300  54.006199        0.0       12.0  1339632000      83   
 2      14.199500  54.006302        0.0       12.0  1402876800      86   
 3      14.201000  54.006500        0.0       12.0  1278460800      91   
 4      14.200500  54.006668        0.0       12.0  1309910400     101   
 ...          ...        ...        ...        ...         ...     ...   
 20237  20.030001  57.343300        0.0      236.0   524620800   14175   
 20238  21.500000  59.433300        0.0      156.0   555033600   15712   
 20239  21.525801  59.439800        0.0      160.0   585446400   15840   
 20240  21.525801  59.439800      150.0      160.0   585446400   15847   
 20241  23.555000  65.239998        0.0       73.0   587001600   20130   
 
              nuclide      value       _unc  _dl  _sal  _temp  _unit  
 0                 h3   850

In [ ]:
#|eval: false
tfm.dfs['sediment']

,lon,lat,tot_depth,time,sed_type,sample,nuclide,value,_unc,_dl,_unit
0,10.850000,54.049999,22.0,866592000,58,842,be7,24.299999,7.7760,1,4
1,10.203300,54.415001,13.0,811641600,58,4064,be7,45.500000,4.5500,1,4
2,10.203300,54.415001,13.0,811641600,58,4069,be7,7.000000,NaN,2,4
3,10.203300,54.415001,13.0,811641600,58,4074,be7,4.800000,NaN,2,4
4,11.750000,54.416698,24.0,838512000,58,4535,be7,6.900000,1.9320,1,4
...,...,...,...,...,...,...,...,...,...,...,...
37084,23.391199,65.277496,90.0,1283299200,2,37039,bi212,42.900002,6.1347,1,4
37085,23.391199,65.277496,90.0,1283299200,2,37048,bi212,58.400002,6.1904,1,4
37086,23.391199,65.277496,90.0,1283299200,2,37065,bi212,51.400002,5.9624,1,4
37087,23.391199,65.277496,90.0,1283299200,2,37074,bi212,41.799999,5.4758,1,4


***

### Format: Time

In [ ]:
#| export
class LookupTimeFromEncodedTime(Callback):
    def __init__(self, cfg): fc.store_attr()
    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            tfm.dfs[grp]['Sampling start date'] = tfm.dfs[grp]['time'].apply(lambda row: self.format_date(row)) 
            tfm.dfs[grp]['Sampling start time'] = tfm.dfs[grp]['time'].apply(lambda row: self.format_time(row))
    
    
    def format_date(self, x): 
        date_time = num2pydate(x, units=self.cfg['units']['time'])
        date = date_time.strftime('%d-%b-%Y')
        return date
    
    def format_time(self, x): 
        date_time = num2pydate(x, units=self.cfg['units']['time'])
        time = date_time.strftime('%H:%M:%S') 
        return time

In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg())])
tfm()

{'seawater':              lon        lat  smp_depth  tot_depth        time  sample  \
 0      14.199800  54.006001        0.0       11.0  1497744000      78   
 1      14.202300  54.006199        0.0       12.0  1339632000      83   
 2      14.199500  54.006302        0.0       12.0  1402876800      86   
 3      14.201000  54.006500        0.0       12.0  1278460800      91   
 4      14.200500  54.006668        0.0       12.0  1309910400     101   
 ...          ...        ...        ...        ...         ...     ...   
 20237  20.030001  57.343300        0.0      236.0   524620800   14175   
 20238  21.500000  59.433300        0.0      156.0   555033600   15712   
 20239  21.525801  59.439800        0.0      160.0   585446400   15840   
 20240  21.525801  59.439800      150.0      160.0   585446400   15847   
 20241  23.555000  65.239998        0.0       73.0   587001600   20130   
 
              nuclide      value       _unc  _dl  _sal  _temp  _unit  \
 0                 h3   85

In [ ]:
tfm.dfs['seawater']['Sampling start date']

0        18-Jun-2017
1        14-Jun-2012
2        16-Jun-2014
3        07-Jul-2010
4        06-Jul-2011
            ...     
20237    17-Aug-1986
20238    04-Aug-1987
20239    21-Jul-1988
20240    21-Jul-1988
20241    08-Aug-1988
Name: Sampling start date, Length: 20242, dtype: object

***

### Lookup: Sample Type 

In [ ]:
#| export
class GetSampleTypeCB(Callback):
    def __init__(self): fc.store_attr()
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k]['Sample type'] = k.upper()
            

In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB()])
tfm()['biota']['Sample type']

0        BIOTA
1        BIOTA
2        BIOTA
3        BIOTA
4        BIOTA
         ...  
14868    BIOTA
14869    BIOTA
14870    BIOTA
14871    BIOTA
14872    BIOTA
Name: Sample type, Length: 14873, dtype: object

***

### Lookup : Nuclide 

In [ ]:
#| export
def get_nucnames_lut():
    df = pd.read_excel(nuc_lut_path(), usecols=['nc_name','nusymbol'])
    return df.set_index('nc_name').to_dict()['nusymbol']

In [ ]:
# | export
class LookupNuclideByIdCB(Callback):
    "Lookup MARIS nuclide_id."
    def __init__(self,
                 fn_lut=get_nucnames_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['Nuclide'] = tfm.dfs[k]['nuclide'].replace(lut)
            tfm.dfs[k]['Nuclide']=tfm.dfs[k]['Nuclide'].str.strip()
            tfm.dfs[k]['Nuclide']=tfm.dfs[k]['Nuclide'].str.replace(',','_')
            
            

In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(),
                            LookupNuclideByIdCB(),
                            ])
tfm()

{'seawater':              lon        lat  smp_depth  tot_depth        time  sample  \
 0      14.199800  54.006001        0.0       11.0  1497744000      78   
 1      14.202300  54.006199        0.0       12.0  1339632000      83   
 2      14.199500  54.006302        0.0       12.0  1402876800      86   
 3      14.201000  54.006500        0.0       12.0  1278460800      91   
 4      14.200500  54.006668        0.0       12.0  1309910400     101   
 ...          ...        ...        ...        ...         ...     ...   
 20237  20.030001  57.343300        0.0      236.0   524620800   14175   
 20238  21.500000  59.433300        0.0      156.0   555033600   15712   
 20239  21.525801  59.439800        0.0      160.0   585446400   15840   
 20240  21.525801  59.439800      150.0      160.0   585446400   15847   
 20241  23.555000  65.239998        0.0       73.0   587001600   20130   
 
              nuclide      value       _unc  _dl  _sal  _temp  _unit  \
 0                 h3   85

In [ ]:
#|eval: false
tfm.dfs['biota']['Nuclide'].unique()

array(['7Be', '40K', '54Mn', '57Co', '58Co', '60Co', '65Zn', '89Sr',
       '90Sr', '95Zr', '95Nb', '99Tc', '103Ru', '106Ru', '108mAg',
       '110mAg', '124Sb', '125Sb', '129mTe', '131I', '134Cs', '137Cs',
       '140Ba', '140La', '141Ce', '144Ce', '155Eu', '210Pb', '212Pb',
       '214Pb', '214Bi', '210Po', '223Ra', '224Ra', '226Ra', '228Ra',
       '228Ac', '228Th', '232Th', '235U', '238Pu', '241Am', '134_137Cs',
       '239_240Pu', '152Eu', '59Fe', '153Gd', '86Rb', '46Sc', '113Sn',
       '117mSn', '208Tl'], dtype=object)

***

### Format : Longitude and Latitude 

Convert from Longitude and Latitude DDD.DDDDD° to degrees, minutes, seconds and direction.

In [ ]:
# | export
def deg_to_dms(deg, coordinate='lat'):
    """Convert from decimal degrees to degrees, minutes, seconds."""
    m, s = divmod(abs(deg)*3600, 60)
    d, m = divmod(m, 60)
    
    if deg < 0:
        if coordinate == 'lat':
            cord = 'S'
        elif coordinate == 'lon':
            cord = 'W'
    else:
        if coordinate == 'lat':
            cord = 'N' 
        elif coordinate == 'lon':
            cord = 'E'                       
        
    d, m = int(d), int(m)
    
    return pd.Series([d, m, s, cord])

In [ ]:
# | export
class ConvertLonLatCB(Callback):
    "Convert from Longitude and Latitude DDD.DDDDD° to degrees, minutes, seconds and direction."
    def __init__(self, fn_convert=deg_to_dms):
        fc.store_attr()

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            tfm.dfs[grp][['Latitude degrees','Latitude minutes','Latitude seconds','Latitude direction']] = tfm.dfs[grp]['lat'].apply(self.fn_convert, coordinate='lat')
            tfm.dfs[grp][['Longitude degrees','Longitude minutes','Longitude seconds','Longitude direction']] = tfm.dfs[grp]['lon'].apply(self.fn_convert, coordinate='lon')


In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(),
                            LookupNuclideByIdCB(),
                            ConvertLonLatCB()
                            ])
tfm()

{'seawater':              lon        lat  smp_depth  tot_depth        time  sample  \
 0      14.199800  54.006001        0.0       11.0  1497744000      78   
 1      14.202300  54.006199        0.0       12.0  1339632000      83   
 2      14.199500  54.006302        0.0       12.0  1402876800      86   
 3      14.201000  54.006500        0.0       12.0  1278460800      91   
 4      14.200500  54.006668        0.0       12.0  1309910400     101   
 ...          ...        ...        ...        ...         ...     ...   
 20237  20.030001  57.343300        0.0      236.0   524620800   14175   
 20238  21.500000  59.433300        0.0      156.0   555033600   15712   
 20239  21.525801  59.439800        0.0      160.0   585446400   15840   
 20240  21.525801  59.439800      150.0      160.0   585446400   15847   
 20241  23.555000  65.239998        0.0       73.0   587001600   20130   
 
              nuclide      value       _unc  _dl  ...  Sample type    Nuclide  \
 0               

In [ ]:
#|eval: false
tfm.dfs['seawater'].columns

Index(['lon', 'lat', 'smp_depth', 'tot_depth', 'time', 'sample', 'nuclide',
       'value', '_unc', '_dl', '_sal', '_temp', '_unit', 'Sampling start date',
       'Sampling start time', 'Sample type', 'Nuclide', 'Latitude degrees',
       'Latitude minutes', 'Latitude seconds', 'Latitude direction',
       'Longitude degrees', 'Longitude minutes', 'Longitude seconds',
       'Longitude direction'],
      dtype='object')

***

### Lookup : Units

In [ ]:
#| export
def get_unitnames_lut():
    df = pd.read_excel(unit_lut_path(), usecols=['unit_id','unit'])
    return df.set_index('unit_id').to_dict()['unit']

In [ ]:
# | export
class LookupUnitByIdCB(Callback):
    "Lookup MARIS unit by unit_id."
    def __init__(self,
                 fn_lut=get_unitnames_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['Unit'] = tfm.dfs[k]['_unit'].replace(lut)
                        

In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(),
                            LookupNuclideByIdCB(),
                            ConvertLonLatCB(), 
                            LookupUnitByIdCB()
                            ])
tfm()

{'seawater':              lon        lat  smp_depth  tot_depth        time  sample  \
 0      14.199800  54.006001        0.0       11.0  1497744000      78   
 1      14.202300  54.006199        0.0       12.0  1339632000      83   
 2      14.199500  54.006302        0.0       12.0  1402876800      86   
 3      14.201000  54.006500        0.0       12.0  1278460800      91   
 4      14.200500  54.006668        0.0       12.0  1309910400     101   
 ...          ...        ...        ...        ...         ...     ...   
 20237  20.030001  57.343300        0.0      236.0   524620800   14175   
 20238  21.500000  59.433300        0.0      156.0   555033600   15712   
 20239  21.525801  59.439800        0.0      160.0   585446400   15840   
 20240  21.525801  59.439800      150.0      160.0   585446400   15847   
 20241  23.555000  65.239998        0.0       73.0   587001600   20130   
 
              nuclide      value       _unc  _dl  ...    Nuclide  \
 0                 h3   850.00

In [ ]:
#|eval: false
tfm.dfs['seawater'].columns

Index(['lon', 'lat', 'smp_depth', 'tot_depth', 'time', 'sample', 'nuclide',
       'value', '_unc', '_dl', '_sal', '_temp', '_unit', 'Sampling start date',
       'Sampling start time', 'Sample type', 'Nuclide', 'Latitude degrees',
       'Latitude minutes', 'Latitude seconds', 'Latitude direction',
       'Longitude degrees', 'Longitude minutes', 'Longitude seconds',
       'Longitude direction', 'Unit'],
      dtype='object')

***

### Lookup : Value type (_dl) 


In [ ]:
#| export
def get_detectionlimitnames_lut():
    df = pd.read_excel(detection_limit_lut_path(), usecols=['id','name'])
    return df.set_index('id').to_dict()['name']

In [ ]:
# | export
class LookupValueTypeByIdCB(Callback):
    "Lookup MARIS Value Type."
    def __init__(self,
                 fn_lut=get_detectionlimitnames_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['Value type'] = tfm.dfs[k]['_dl'].replace(lut)
                        

In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(),
                            LookupNuclideByIdCB(),
                            ConvertLonLatCB(), 
                            LookupUnitByIdCB(),
                            LookupValueTypeByIdCB()
                            ])
tfm()

{'seawater':              lon        lat  smp_depth  tot_depth        time  sample  \
 0      14.199800  54.006001        0.0       11.0  1497744000      78   
 1      14.202300  54.006199        0.0       12.0  1339632000      83   
 2      14.199500  54.006302        0.0       12.0  1402876800      86   
 3      14.201000  54.006500        0.0       12.0  1278460800      91   
 4      14.200500  54.006668        0.0       12.0  1309910400     101   
 ...          ...        ...        ...        ...         ...     ...   
 20237  20.030001  57.343300        0.0      236.0   524620800   14175   
 20238  21.500000  59.433300        0.0      156.0   555033600   15712   
 20239  21.525801  59.439800        0.0      160.0   585446400   15840   
 20240  21.525801  59.439800      150.0      160.0   585446400   15847   
 20241  23.555000  65.239998        0.0       73.0   587001600   20130   
 
              nuclide      value       _unc  _dl  ...  Latitude degrees  \
 0                 h3  

***

### Lookup : Biogroup

Biogroup is in netcdf but not in OPEN REfINE csv format.  Should we include this in Netcdf? 

***

### Lookup : Species

In [ ]:
#| export
def get_species_lut():
    df = pd.read_excel(species_lut_path(), usecols=['species_id','species'])
    return df.set_index('species_id').to_dict()['species']

In [ ]:
# | export
class LookupSpeciesByIdCB(Callback):
    "Lookup MARIS species by species_id."
    def __init__(self,
                 fn_lut=get_species_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            if 'species' in tfm.dfs[k].columns: 
                tfm.dfs[k]['Species'] = tfm.dfs[k]['species'].replace(lut)
                        

In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(),
                            LookupNuclideByIdCB(),
                            ConvertLonLatCB(), 
                            LookupUnitByIdCB(),
                            LookupValueTypeByIdCB(),
                            LookupSpeciesByIdCB()
                            ])
tfm()

{'seawater':              lon        lat  smp_depth  tot_depth        time  sample  \
 0      14.199800  54.006001        0.0       11.0  1497744000      78   
 1      14.202300  54.006199        0.0       12.0  1339632000      83   
 2      14.199500  54.006302        0.0       12.0  1402876800      86   
 3      14.201000  54.006500        0.0       12.0  1278460800      91   
 4      14.200500  54.006668        0.0       12.0  1309910400     101   
 ...          ...        ...        ...        ...         ...     ...   
 20237  20.030001  57.343300        0.0      236.0   524620800   14175   
 20238  21.500000  59.433300        0.0      156.0   555033600   15712   
 20239  21.525801  59.439800        0.0      160.0   585446400   15840   
 20240  21.525801  59.439800      150.0      160.0   585446400   15847   
 20241  23.555000  65.239998        0.0       73.0   587001600   20130   
 
              nuclide      value       _unc  _dl  ...  Latitude degrees  \
 0                 h3  

***

### Lookup : Body part

In [ ]:
#| export
def get_bodypart_lut():
    df = pd.read_excel(bodyparts_lut_path(), usecols=['bodypar_id','bodypar'])
    return df.set_index('bodypar_id').to_dict()['bodypar']

In [ ]:
# | export
class LookupBodypartByIdCB(Callback):
    "Lookup MARIS bodypart by bodypar_id."
    def __init__(self,
                 fn_lut=get_bodypart_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            if 'body_part' in tfm.dfs[k].columns: 
                tfm.dfs[k]['Body part'] = tfm.dfs[k]['body_part'].replace(lut)
                        

In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(),
                            LookupNuclideByIdCB(),
                            ConvertLonLatCB(), 
                            LookupUnitByIdCB(),
                            LookupValueTypeByIdCB(),
                            LookupSpeciesByIdCB(),
                            LookupBodypartByIdCB()
                            ])
tfm()

{'seawater':              lon        lat  smp_depth  tot_depth        time  sample  \
 0      14.199800  54.006001        0.0       11.0  1497744000      78   
 1      14.202300  54.006199        0.0       12.0  1339632000      83   
 2      14.199500  54.006302        0.0       12.0  1402876800      86   
 3      14.201000  54.006500        0.0       12.0  1278460800      91   
 4      14.200500  54.006668        0.0       12.0  1309910400     101   
 ...          ...        ...        ...        ...         ...     ...   
 20237  20.030001  57.343300        0.0      236.0   524620800   14175   
 20238  21.500000  59.433300        0.0      156.0   555033600   15712   
 20239  21.525801  59.439800        0.0      160.0   585446400   15840   
 20240  21.525801  59.439800      150.0      160.0   585446400   15847   
 20241  23.555000  65.239998        0.0       73.0   587001600   20130   
 
              nuclide      value       _unc  _dl  ...  Latitude degrees  \
 0                 h3  

***

### Lookup : Sediment type

In [ ]:
#| export
def get_sediments_lut():
    df = pd.read_excel(sediments_lut_path(), usecols=['sedtype_id','sedtype'])
    return df.set_index('sedtype_id').to_dict()['sedtype']

In [ ]:
# | export
class LookupSedimentTypeByIdCB(Callback):
    "Lookup MARIS sedtype by sedtype_id."
    def __init__(self,
                 fn_lut=get_sediments_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            if 'sed_type' in tfm.dfs[k].columns: 
                tfm.dfs[k]['Sediment type'] = tfm.dfs[k]['sed_type'].replace(lut)
                        

In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(),
                            LookupNuclideByIdCB(),
                            ConvertLonLatCB(), 
                            LookupUnitByIdCB(),
                            LookupValueTypeByIdCB(),
                            LookupSpeciesByIdCB(),
                            LookupBodypartByIdCB(),
                            LookupSedimentTypeByIdCB()
                            ])
tfm()

{'seawater':              lon        lat  smp_depth  tot_depth        time  sample  \
 0      14.199800  54.006001        0.0       11.0  1497744000      78   
 1      14.202300  54.006199        0.0       12.0  1339632000      83   
 2      14.199500  54.006302        0.0       12.0  1402876800      86   
 3      14.201000  54.006500        0.0       12.0  1278460800      91   
 4      14.200500  54.006668        0.0       12.0  1309910400     101   
 ...          ...        ...        ...        ...         ...     ...   
 20237  20.030001  57.343300        0.0      236.0   524620800   14175   
 20238  21.500000  59.433300        0.0      156.0   555033600   15712   
 20239  21.525801  59.439800        0.0      160.0   585446400   15840   
 20240  21.525801  59.439800      150.0      160.0   585446400   15847   
 20241  23.555000  65.239998        0.0       73.0   587001600   20130   
 
              nuclide      value       _unc  _dl  ...  Latitude degrees  \
 0                 h3  

***

### Rename columns

In [ ]:
#| export
# Define columns of interest (keys) and renaming rules (values).
def get_renaming_rules_netcdf2OpenRefine():
    vars = cdl_cfg()['vars']
    return {('seawater','biota', 'sediment') : {    
                                                        ## DEFAULT
                                                        'Sample type' : 'Sample type',
                                                        'Latitude degrees' : 'Latitude degrees',
                                                        'Latitude minutes' : 'Latitude minutes',
                                                        'Latitude seconds' : 'Latitude seconds',
                                                        'Latitude direction' : 'Latitude direction',
                                                        'Longitude degrees' : 'Longitude degrees',
                                                        'Longitude minutes' : 'Longitude minutes',
                                                        'Longitude seconds' : 'Longitude seconds', 
                                                        'Longitude direction' : 'Longitude direction', 
                                                        vars['defaults']['lat']['name'] : 'Latitude decimal',
                                                        vars['defaults']['lon']['name'] : 'Longitude decimal',
                                                        'Sampling start date' : 'Sampling start date',
                                                        'Sampling start time' : 'Sampling start time',
                                                        'Nuclide' : 'Nuclide',
                                                        'Value type': 'Value type',
                                                        'Unit' : 'Unit',
                                                        'value' : 'Activity or MDA',
                                                        vars['suffixes']['uncertainty']['name'] : 'Uncertainty',
                                                        #'data_provider_station_id' : 'Station ID',
                                                        #vars['defaults']['data_provider_sample_id']['name'] :'Sample ID',
                                                        #'profile_id' : 'Profile or transect ID',                                                        
                                                        #'Sampling method' : 'sampling_method'
                                                        #'Preparation method' : 'preparation_method'
                                                        #'Counting method' : 'counting_method'
                                                        #'Sample notes' : 'sample_notes'
                                                        #'Measurement notes' : 'measurement_notes'
                                                    },
                  ('seawater',) : {
                                ## SEAWATER
                                vars['defaults']['tot_depth']['name'] : 'Total depth',
                                vars['defaults']['smp_depth']['name'] : 'Sampling depth' ,
                                vars['suffixes']['salinity']['name'] : 'Salinity',
                                vars['suffixes']['temperature']['name'] : 'Temperature',
                                #vars['suffixes']['filtered']['name'] : 'Filtered' TODO: Include in NetCDF encoder. 
                                },
                  ('biota',) : { 
                                ## BIOTA
                                'Species' : 'Species',
                                'Body part' : 'Body part',
                                'bio_group' : vars['bio']['bio_group']['name'],
                                #'SDEPTH' : vars['defaults']['smp_depth']['name'],
                                #'dry_wet_ratio' : 'Dry/wet ratio'
                                #'Drying Method' : drying_method
                                
                                },
                  ('sediment',) : {
                                ## SEDIMENT
                                vars['defaults']['tot_depth']['name'] : 'Total depth',
                                'Sediment type' : 'Sediment type',
                                #'top' : 'Top',
                                #'bottom' : 'Bottom', 
                                #'dry_wet_ratio' : 'Dry/wet ratio'
                                #'Drying Method' : drying_method
                                }
                    }

In [ ]:
class SelectAndRenameColumnCB(Callback):
    def __init__(self,
                 fn_renaming_rules,
                ):
        fc.store_attr()
    def __call__(self, tfm):
        renaming = self.fn_renaming_rules()
        for grp in tfm.dfs.keys():            
            # get columns related to the grp (e.g. 'biota').
            coi = [v for k, v in renaming.items() if grp in k]
            # Join cols of interest
            coi_rename = {}
            for d in coi:
                for k, v in d.items(): 
                    coi_rename[k]=v
            # list cols
            cols = list(coi_rename.keys()) 
            # select cols in df 
            tfm.dfs[grp] = tfm.dfs[grp].loc[:, cols]
            # Rename cols
            tfm.dfs[grp].rename(columns=coi_rename, inplace=True)

In [ ]:
#|eval: false
dfs = netcdf4_to_df(fname_in)
tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(),
                            LookupNuclideByIdCB(),
                            ConvertLonLatCB(), 
                            LookupUnitByIdCB(),
                            LookupValueTypeByIdCB(),
                            LookupSpeciesByIdCB(),
                            LookupBodypartByIdCB(),
                            LookupSedimentTypeByIdCB(),
                            SelectAndRenameColumnCB(get_renaming_rules_netcdf2OpenRefine)
                            ])
tfm()

{'seawater':       Sample type  Latitude degrees  Latitude minutes  Latitude seconds  \
 0        SEAWATER                54                 0         21.601868   
 1        SEAWATER                54                 0         22.315979   
 2        SEAWATER                54                 0         22.686768   
 3        SEAWATER                54                 0         23.400879   
 4        SEAWATER                54                 0         24.005127   
 ...           ...               ...               ...               ...   
 20237    SEAWATER                57                20         35.879517   
 20238    SEAWATER                59                25         59.880066   
 20239    SEAWATER                59                26         23.280945   
 20240    SEAWATER                59                26         23.280945   
 20241    SEAWATER                65                14         23.992310   
 
       Latitude direction  Longitude degrees  Longitude minutes  \
 0     

***

### Encoding

In [ ]:
#| export
def encode(fname_in, fname_out, ref_id=-1, **kwargs):
    dfs = netcdf4_to_df(fname_in)
    tfm = Transformer(dfs, cbs=[ReshapeWideToLong(),
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(),
                            LookupNuclideByIdCB(),
                            ConvertLonLatCB(), 
                            LookupUnitByIdCB(),
                            LookupValueTypeByIdCB(),
                            LookupSpeciesByIdCB(),
                            LookupBodypartByIdCB(),
                            LookupSedimentTypeByIdCB(),
                            SelectAndRenameColumnCB(get_renaming_rules_netcdf2OpenRefine)
                            ])
    
    encoder = OpenRefineCsvEncoder(tfm(), 
                            dest_fname=fname_out,
                            ref_id = ref_id,
                            **kwargs)
    encoder.encode()
    return encoder

In [ ]:
encode(fname_in, fname_out, ref_id, verbose=False)

<marisco.serializers.OpenRefineCsvEncoder>

## TODO

TODO Review Maris Nuclides lut. Cs127?

TODO: Should the var be called 'detection limit'? Is 'value type' more appropriate?

TODO Biogroup not used in OPEN REfINE csv format. Confirm this

TODO Ask about Species dbo. Paul said there is a larger one. 

TODO: Maintain sample (i.e. index) in the output. 